# Création fichier csv clean

Ce notebook est consacré à la création du fichier qui sera utilisé comme base de données. Pour cela, nou savons pris le format CSV, avec comme séparateur la barre verticale "|" 
Il est séparé en plusieurs étapes : 

    1) [Installations et importations](#petit)
    2) Fonctions utilitaires
    3) Script permettant l'écriture du fichier
    

## 1) Installations et importations {#petit1}

Besoin d'installer

In [ ]:
#Installations pour la récupération des données

!pip install webdriver-manager
!pip install -q lxml
!pip install bs4

In [ ]:
#Installation pour le cleaning

!pip install spacy 
!python -m spacy download fr_core_news_sm

In [6]:
#imports

import pandas as pd
import bs4
import lxml
import pandas
import urllib
import re

from urllib import request

import spacy

### 2) Fonctions utilitaires

In [50]:
#Fonctions utilitaires

def pageurl(x) : 
    return "https://questions.assemblee-nationale.fr/q16/16-"+str(x)+"QE.htm"


nlp = spacy.load('fr_core_news_sm')

def clean_docs(texts, ministere, remove_stopwords=True, n_process = 4):
    
    docs = nlp.pipe(texts, 
                    n_process=n_process,
                    disable=['parser', 'ner', 'textcat'])
    stopwords = nlp.Defaults.stop_words.union(set(['m.','mme','ministre', ministere.lower()]))

    docs_cleaned = []
    for doc in docs:
        tokens = [tok.text.lower().strip() for tok in doc if not tok.is_punct]
        if remove_stopwords:
            tokens = [tok for tok in tokens if tok not in stopwords]
        doc_clean = ' '.join(tokens)
        docs_cleaned.append(doc_clean)
        
    return docs_cleaned

# pas sur de l'utilité : mots qui peuvent etre quand meme utile (souveraineté ?), appelle parfois le délégué des finances alors qu'interroge le ministere des comptes publics
"""def clean_question2(text, ministere, nom, remove_stopwords=True, n_process=4):
    
    text = nlp(text)

    inclus = ministere.lower().split() + nom.lower().split()
    print(inclus)
    stopwords = nlp.Defaults.stop_words 
    stopwords = stopwords.union(set(['m.','mme','ministre']+inclus))
    
    tokens = [tok.text.lower().strip() for tok in text if not tok.is_punct]
    if remove_stopwords:
        tokens = [tok for tok in tokens if tok not in stopwords]
    question_clean = ' '.join(tokens)
    return question_clean"""

def clean_question(text, nom, remove_stopwords=True, n_process=4):
    
    text = nlp(text)

    stopwords = nlp.Defaults.stop_words 
    stopwords = stopwords.union(set(['m.','mme','ministre']+nom.lower().split()))
    
    tokens = [tok.text.lower().strip() for tok in text if not tok.is_punct]
    if remove_stopwords:
        tokens = [tok for tok in tokens if tok not in stopwords]
    question_clean = ' '.join(tokens)
    return question_clean



In [57]:
#création fichier

données = open("data_cleaned.csv", "w", encoding="utf-8")

données.write("id|groupe|nom|ministère_interrogé|rubrique|titre|date|question|question_clean")

données.close()

In [58]:
données = open("data_cleaned.csv", "a", encoding="utf-8")

for k in range(500, 1500) : 
    source_code = request.urlopen(pageurl(k)).read()
    page = bs4.BeautifulSoup(source_code)
    question0 = page.find("div", {"class" : "question"})
    question = question0.find("p")
    question = question.text.strip()
    
    rubrique0 = page.findAll("div", {"class" : "question_col33"})
    rubrique,titre = rubrique0[0].find("p"), rubrique0[1].find("p")
    rubrique, titre = rubrique.text.strip()[11:], titre.text.strip()[8:]
    
    ministere0 = page.find("div", {"class" : "question_col50"})
    ministere = ministere0.text.strip()[23:]

    lien0 = page.find("span", {"class" : "question_big_content"})
    lien = lien0.find("a").get("href")

    date0 = page.find("div", {"class" : "question_publish_date"})
    date = date0.find("span")
    date = date.text.strip()
    
    source_code_parlementaire = request.urlopen(lien).read()
    page_parlementaire = bs4.BeautifulSoup(source_code_parlementaire)

    groupe0 = page_parlementaire.find("a", {"class" : "h4 _colored link"})
    groupe = groupe0.text.strip()

    nom0 = page_parlementaire.find("h1", {"class" : "h1 _mt-small"})
    nom = nom0.text.strip()

    question_clean = clean_question(question, nom)


    données.write("\n" + str(k) + "|" + groupe + "|" + nom +  "|" + ministere + "|" + rubrique + "|" + titre + "|" + date + "|" + question + "|" + question_clean)

données.close()